# Keras MNIST Guided Example

In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


## Keras for neural networks - guided example
- Digit recognition
    - classic machine learning problem
    - MNIST dataset
- Create 3 different styles of networks, discuss advanages and disadvantages

## MNIST Data 
- values of pixels
- not a single table, different, higher dimensionality structure

In [2]:
# import dataset
from keras.datasets import mnist

# Import various components for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 9s 1us/step


In [4]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

### Multi Layer Perceptron
- set of perceptron models organized into layers
- one layer feeds the next
- Need to reshape data into flat vectors for each digit
- conver outcome to matrix of binary variables, rather than the digit


In [5]:
# Change shape
# Note that our images are 28x28 pixels, so in reshaping to arrays we want
# 60000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


- Create model
- Use dense layers and dropouts
    - dropout drops a certain portion of our perceptrons in order to prevent overfitting
    - actitation function, relu, Rectified Linear Unit
- Using 64 perceptron wide layers, relatively arbitrary, units with 2^x parallelize more efficiently.
- Our number of parameters is the product of our input width plus one and our layer width

In [7]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the
# first layer
# Relu is the activation function
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.4340 - acc: 0.8714 - val_loss: 0.1954 - val_acc: 0.9401
Epoch 2/10
60000/60000 [==============================] - 1s 20us/step - loss: 0.2033 - acc: 0.9393 - val_loss: 0.1417 - val_acc: 0.9582
Epoch 3/10
60000/60000 [==============================] - 1s 19us/step - loss: 0.1565 - acc: 0.9529 - val_loss: 0.1146 - val_acc: 0.9643
Epoch 4/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.1305 - acc: 0.9602 - val_loss: 0.1053 - val_acc: 0.9681
Epoch 5/10
60000/60000 [==============================] - 1s 19us/step - loss: 0.1148 - acc: 0.9651 - val_loss: 0.0998 - val_acc: 0.9712
Epoch 6/10
60000/60000 [==============================] - 1s 18us/step - loss: 0.1040 - acc: 0.9678 - val_loss: 0.0962 - val_acc: 0.9722
Epoch 7/10
60000/60000 [==============================] - 1s 19us/step - loss: 0

### Convolutional Neural Networks
- This code will take serious time to run
- Convolution 
    - takes your data and creates overlapping subsegments testing for a given feature
    - first define shape of data
        - 2d for our example since images are 2d
    - create tiles, called kernels
        - like little windows look over subsets of the data of a given size
        - example below has 3x3 kernels (convolutional layer)
        - example below has 2
    - pooling layer
        - downsampling technique
        - reduces sample size and simplifies later processes
        - looks over grid in non-overlapping segments
    - after pooling flatten data back out so it can be put into dense layers

In [9]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 102s 2ms/step - loss: 0.2654 - acc: 0.9177 - val_loss: 0.0610 - val_acc: 0.9808
Epoch 2/10
60000/60000 [==============================] - 100s 2ms/step - loss: 0.0901 - acc: 0.9738 - val_loss: 0.0436 - val_acc: 0.9846
Epoch 3/10
60000/60000 [==============================] - 99s 2ms/step - loss: 0.0659 - acc: 0.9807 - val_loss: 0.0363 - val_acc: 0.9880
Epoch 4/10
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0545 - acc: 0.9838 - val_loss: 0.0337 - val_acc: 0.9886
Epoch 5/10
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0471 - acc: 0.9855 - val_loss: 0.0324 - val_acc: 0.9891
Epoch 6/10
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0403 - acc: 0.9875 - val_loss: 0.0280 - val_acc: 0.9909
Epoch 7/10
60000/60000 [==========================

### Hierarchical Recurrent Neural Networks
- Lets data cycle through the network
- Data flows in more than one direction 
- Use recurrent layers and often time distribution
- Examples:
    - https://github.com/keras-team/keras/blob/master/examples/mnist_hierarchical_rnn.py
- Even slower than other neural networks
- LTSM = Long short-term memory
    - applicable to tasks such as unsegmented, connected handwriting, speeech recognition
    - each unit has a cell, input gate, output gate, and forget gate
    - remembers values over an arbitrary time interval

In [12]:
# Training parameters
batch_size = 64
num_classes = 10
epochs = 3

# Embedding Dimensions
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4d for Hierarchical RNN
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 114s 2ms/step - loss: 1.0286 - acc: 0.6523 - val_loss: 0.5607 - val_acc: 0.8168
Epoch 2/3
60000/60000 [==============================] - 106s 2ms/step - loss: 0.4317 - acc: 0.8638 - val_loss: 0.3283 - val_acc: 0.8991
Epoch 3/3
60000/60000 [==============================] - 104s 2ms/step - loss: 0.2568 - acc: 0.9215 - val_loss: 0.2588 - val_acc: 0.9218
Test loss: 0.2588168480843306
Test accuracy: 0.9218


### Drill: 99% MLP
We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or concolutional neural nets?

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(128, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.1))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_111 (Dense)            (None, 128)               100480    
_________________________________________________________________
dropout_77 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_112 (Dense)            (None, 128)               16512     
_________________________________________________________________
dropout_78 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [=======================